In [1]:
%load_ext autoreload
%autoreload 2

In [72]:
import sys
import logging

import numpy as np
import pandas as pd

from neuclease.dvid import DvidInstanceInfo, fetch_split_supervoxel_sizes, fetch_mappings
from neuclease.merge_table import load_supervoxel_sizes, compute_body_sizes

In [29]:
handler = logging.StreamHandler(sys.stdout)
root_logger = logging.getLogger()
root_logger.handlers = []
root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)
logging.getLogger('kafka').setLevel(logging.WARNING)

### Load Original supervoxel sizes

In [36]:
orig_sizes_file = '/groups/flyem/data/scratchspace/copyseg-configs/labelmaps/hemibrain/8nm/compute-8nm-extended-fixed-STATS-ONLY-20180402.192015/supervoxel-sizes.h5'
%time orig_sv_sizes = load_supervoxel_sizes(orig_sizes_file)
total_voxel_count = orig_sv_sizes.sum()

Volume contains 188243164 supervoxels and 22.5 Teravoxels in total
CPU times: user 748 ms, sys: 1.11 s, total: 1.85 s
Wall time: 7.58 s


### Fetch Supervoxel split fragment sizes

In [73]:
instance_info = DvidInstanceInfo('emdata3:8900', '52f9', 'segmentation')

In [64]:
split_fragment_sizes = fetch_split_supervoxel_sizes(instance_info, include_retired=True)

Reading kafka messages from kafka.int.janelia.org:9092 for emdata3:8900 / 52f9 / segmentation
Reading 155276 kafka messages took 7.97636866569519 seconds
Fetching sizes for 5843 split supervoxels...
Fetching sizes for 5843 split supervoxels took 0:00:07.994808


### Combine old/split sizes

In [65]:
all_sizes = pd.concat((orig_sv_sizes, split_fragment_sizes))
all_sizes = all_sizes[~all_sizes.index.duplicated(keep='last')]

In [66]:
assert all_sizes.index.dtype == np.uint64
assert all_sizes.dtype == np.uint32
all_sizes.shape[0] - orig_sv_sizes.shape[0]

7460

### Fetch mapping

In [51]:
retired_supervoxels = all_sizes[all_sizes == 0].index
mapping = fetch_mappings(instance_info, retired_supervoxels=retired_supervoxels)

Fetching http://emdata3:8900/api/node/52f9/segmentation/mappings...
Fetching http://emdata3:8900/api/node/52f9/segmentation/mappings took 0:00:31.064037
Parsing mapping...
Parsing mapping took 0:00:08.105745
Appending missing identity-mappings...
Appending missing identity-mappings took 0:00:22.367342


### Compute body sizes

In [67]:
body_sizes = compute_body_sizes(all_sizes, mapping)

Dropping unknown supervoxels
Applying sizes to mapping
Aggregating sizes by body


In [74]:
body_sizes.to_csv(f'/nrs/flyem/bergs/finish-02/body-sizes-{instance_info.uuid[:4]}.csv')